# Derivation of a Quasi-Quartic Derivation of Divergence
## Plan
- Find a finite difference approximation for $\partial\Psi/\partial x$
- Express this as a differences of fluxes
- Express using local gradients

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as sy
from sympy import latex
from sympy.matrices import Matrix, MatrixSymbol
from fractions import Fraction as Fr
from sympy import I, E, pi, Q

## Quartic finite difference approximation for $\partial\Psi/\partial x$
Assume that $\Psi$ follows a quartic polynomial:
\begin{eqnarray}
\psi &=& ax^4 + bx^3 + cx^2 + dx + e
\end{eqnarray}
with $\Delta x=1$ and $x=0$ at $j=0$. We know solutions
\begin{eqnarray}
\psi_{j-3} = \psi_0 &=& (-3)^4 a + (-3)^3 b + (-3)^2 c - 3d + e \\
\psi_{j-2} = \psi_1 &=& (-2)^4 a + (-2)^3 b + (-2)^2 c - 2d + e \\
\psi_{j-1} = \psi_2 &=& a - b + c - d  + e \\
\psi_{j}   = \psi_3 &=& e \\
\psi_{j+1} = \psi_4 &=& a + b + c + d + e  \\
\end{eqnarray}
So $(a,b,c,d,e)$ is the solution of the matrix equation
\begin{equation*}
\left( \begin{array}{ccccc}
3^4 & -3^3 & 3^2 & -3 & 1 \\
2^4 & -2^3 & 2^2 & -2 & 1 \\
1 & -1 & 1 & -1 & 1 \\
0 & 0 & 0 & 0 & 1 \\
1 & 1 & 1 & 1 & 1 & 1 \\
\end{array} \right) 
\left( \begin{array}{c}
a \\ b \\ c \\ d \\ e
\end{array} \right) 
=
\left( \begin{array}{c}
\psi_0 \\ \psi_1 \\ \psi_2 \\ \psi_3 \\ \psi_4
\end{array} \right) 
\end{equation*}
We only need an expression for 
\begin{equation*}
\partial\Psi/\partial x_j = e
\end{equation*}

In [ ]:
# Find the finite differene gradient
# Coefficients a,b,c,d,e of the polynomial
PolyCoeffs = sy.Matrix(sy.symarray("PolyCoeffs", (5,)))
# Grid point values at j-3,j-2, j-1, j and j+1 
Psi = sy.Matrix(sy.symarray("Ψ", (5,)), real=True)
Dx = sy.symbols("Δx", real=True)
polyM = Matrix([[3**4, -3**3, 3**2,-3,1],
                [2**4, -2**3, 2**2,-2,1],
                [1,-1,1,-1,1],
                [0,0,0,0,1],
                [1,1,1,1,1]])
PolyCoeffs = polyM.solve(Psi)
Ddx = PolyCoeffs[3]/Dx
Ddx
#print(latex(Ddx))

In [ ]:
# Express as a difference of fluxes
# Find Ψ_l and Ψ_r so that Ddx = (Ψ_r - Ψ_l)/dx
DdxR = Ddx*Dx # The residual of Ddx*Dx after removal of (Ψ_r - Ψ_l)
[Psil, Psir] = sy.symbols("Ψ_l, Ψ_r", real=True)
Psil = Psir = sy.S.Zero
for j in range(len(Psi)-1):
    Psil = Psil - Psi[j]*DdxR.coeff(Psi[j])
    Psir = Psir - Psi[j+1]*DdxR.coeff(Psi[j])
    DdxR = sy.collect(sy.expand(Ddx*Dx - (Psir - Psil)), Psi)

print('Residual =', DdxR, '\nThe right flux is')
Psir

In [ ]:
# Express $\Psi_r$ as a linear combination of gradients
u = Psi[3]
#grad4u = (Psi[1] - 4*Psi[2] + 6*Psi[3] - 4*Psi[4] + Psi[5])/Dx**4
#grad2u = (Psi[4] + Psi[2] - 2*Psi[3])/Dx**2
#grad2d = (Psi[5] + Psi[3] - 2*Psi[4])/Dx**2

gradGradu = (Psi[1] - Psi[2] - Psi[3] + Psi[4])/(2*Dx**2)
#gradd = (Psi[5] - Psi[3])/(2*Dx)
gradu = (Psi[4] - Psi[2])/(2*Dx)
gradf = (Psi[4] - Psi[3])/Dx

#PsirG = u + gradf/2 - grad2u/12 - grad2d/12 + grad4u/30
#PsirG = u + Dx*gradf/2 - Dx*gradd/6 + Dx*gradu/6 + Dx**4*grad4u/30
PsirG = u - Dx*gradf/6 + 2*Dx*gradu/3 + 2*Dx**2*gradGradu/12
resid = PsirG - Psir
resid

In [ ]:
# Stability Analysis
# mu and eta for quasi-quintic
kDx = sy.symbols("kΔx", real=True)
mu = 1 - E**(-I*kDx)
eta = -mu

indicies = [-2,-1,0,-1]
weights = [sy.Rational(1,12), sy.Rational(-5,12), sy.Rational(13/12), sy.Rational(1/4)]
for j,w in zip(indicies, weights):
    eta += w*(E**(j*I*kDx) - E**((j-1)*I*kDx))

In [ ]:
# RK3 Butcher Tableau
quarter = sy.Rational(1,4)
sixth = sy.Rational(1,6)
half  = sy.Rational(1,2)
RK3 = [[1,0,0], [quarter,quarter,0], [sixth, sixth, 4*sixth]]
RK1 = [[1]]
RK = RK3

In [ ]:
# Amplification factors for Strang carry-over AdImEx with RK3
c, alpha, beta, gamma = sy.symbols("c, alpha, beta, gamma", real=True, positive=True)
A = sy.Matrix(sy.symarray("A", (len(RK)+2,)))
A[0] = 1 - c*(1-alpha)*beta*mu
for i in range(1,len(RK)+1):
    A[i] = A[0]
    for j in range(0,i):
        A[i] -= c*((1-beta)*mu + gamma*eta)*RK[i-1][j]*A[j]
A[-1] = A[-2]/(1 + c*alpha*beta*mu)

In [ ]:
# Amplification factor for beta =0, alpha=0, gamma=1
magAexp = A[-1].subs({alpha:0, beta:0, gamma:1})
magAexp = (sy.re(magAexp)**2 + sy.im(magAexp)**2)**(1/2)

In [ ]:
# Convert to a meshgrid as a funciton of c and kdx
def magA(cs, kdxs):
    try:
        cs, kdxs = np.meshgrid(cs, kdxs)
    except:
        pass
    return sy.lambdify([c, kDx], magAexp**.5, 'numpy')(cs, kdxs)

cs = np.linspace(0,1.5, 16)
kdxs = np.linspace(0, np.pi, 11)
AE = magA(cs, kdxs)

In [ ]:
plt.contourf(cs, kdxs, AE)
plt.colorbar()
plt.contour(cs, kdxs, AE, [1,100], colors='w')
plt.xlabel('c')
plt.ylabel(r'$k\Delta x$')
plt.show()